In [1]:
#C321079 김동하 스마트제조 총괄생산계획

from pyomo.environ import *

def setup_model(D, type_mp="LP"):

    TH = len(D)  
    TIME = range(0, TH+1)  
    T = range(1, TH+1)

    if type_mp == "IP":
        type_var = NonNegativeIntegers
    else:
        type_var = NonNegativeReals

    
    m = ConcreteModel()


    m.W = Var(TIME, domain=type_var, bounds=(0, None))
    m.H = Var(TIME, domain=type_var, bounds=(0, None))
    m.L = Var(TIME, domain=type_var, bounds=(0, None))
    m.P = Var(TIME, domain=type_var, bounds=(0, None))
    m.I = Var(TIME, domain=type_var, bounds=(0, None))
    m.S = Var(TIME, domain=type_var, bounds=(0, None))
    m.C = Var(TIME, domain=type_var, bounds=(0, None))
    m.O = Var(TIME, domain=type_var, bounds=(0, None))

    m.Cost = Objective(
        expr=sum(
            640*m.W[t] + 6*m.O[t] + 300*m.H[t] + 500*m.L[t]
            + 2*m.I[t] + 5*m.S[t] + 10*m.P[t] + 30*m.C[t]
            for t in T),
        sense=minimize)
    
    m.labor = Constraint(T,rule = lambda m, t: m.W[t] == m.W[t-1]+m.H[t]-m.L[t])
    m.capacity = Constraint(T,rule = lambda m, t: m.I[t] <= 40*m.W[t]+0.25*m.O[t])
    m.inventory = Constraint(T,rule = lambda m, t: m.I[t] == m.I[t-1]+m.P[t]+m.C[t]-D[t-1]-m.S[t-1]+m.S[t])
    m.overtime = Constraint(T,rule = lambda m, t: m.O[t] <= 10*m.W[t])

    m.W_0 = Constraint(expr=m.W[0] == 80)
    m.I_0 = Constraint(expr=m.I[0] == 1000)
    m.S_0 = Constraint(expr=m.S[0] == 0)

    m.last_inventory = Constraint(expr=m.I[TH] >= 500)
    m.last_shortage = Constraint(expr=m.S[TH] == 0)

    return m

demand=[1600,3000,3200]

model = setup_model(demand,"LP")
model .pprint()


8 Var Declarations
    C : Size=4, Index={0, 1, 2, 3}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals
    H : Size=4, Index={0, 1, 2, 3}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals
    I : Size=4, Index={0, 1, 2, 3}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : 

In [2]:
demand = [1600, 3000, 3200, 3800, 2200, 2200]
T = range(0, len(demand)+1)

model = setup_model(demand, "LP")

solver = SolverFactory('glpk').solve(model).write()

print('Minimal Cost = ', model.Cost())
print('(수요) D = ', [0] + demand)
print('(작업자) W = ', [model.W[t]() for t in T])
print('(고용) H = ', [model.H[t]() for t in T])
print('(해고) L = ', [model.L[t]() for t in T])
print('(생산) P = ', [model.P[t]() for t in T])
print('(재고) I = ', [model.I[t]() for t in T])
print('(부재고) S = ', [model.S[t]() for t in T])
print('(외주) C = ', [model.C[t]() for t in T])
print('(잔업) O = ', [model.O[t]() for t in T])

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 207750.0
  Upper bound: 207750.0
  Number of objectives: 1
  Number of constraints: 29
  Number of variables: 51
  Number of nonzeros: 95
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.03471970558166504
# ----------------------------------------------------------
#   Solution Information
# ----------------------------

In [3]:
demand = [1600, 5000, 3200, 3800, 2200, 2200]
T = range(0, len(demand)+1)

model = setup_model(demand, "IP")

solver = SolverFactory('glpk').solve(model)

print('Minimal Cost = ', model.Cost())
print('(수요) D = ', [0] + demand)
print('(작업자) W = ', [model.W[t]() for t in T])
print('(고용) H = ', [model.H[t]() for t in T])
print('(해고) L = ', [model.L[t]() for t in T])
print('(생산) P = ', [model.P[t]() for t in T])
print('(재고) I = ', [model.I[t]() for t in T])
print('(부재고) S = ', [model.S[t]() for t in T])
print('(외주) C = ', [model.C[t]() for t in T])
print('(잔업) O = ', [model.O[t]() for t in T])

Minimal Cost =  227760.0
(수요) D =  [0, 1600, 5000, 3200, 3800, 2200, 2200]
(작업자) W =  [80.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0]
(고용) H =  [None, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0]
(해고) L =  [None, 80.0, 0.0, 0.0, 0.0, 0.0, 0.0]
(생산) P =  [None, 600.0, 5000.0, 3200.0, 3800.0, 2200.0, 2700.0]
(재고) I =  [1000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 500.0]
(부재고) S =  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
(외주) C =  [None, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
(잔업) O =  [None, 0.0, 0.0, 0.0, 0.0, 0.0, 80.0]


In [4]:
demand = [1600, 5000, 3200, 5800, 2200, 2200, 6500, 2300]
T = range(0, len(demand)+1)

model = setup_model(demand, "IP")

solver = SolverFactory('glpk').solve(model)

print('Minimal Cost = ', model.Cost())
print('(수요) D = ', [0] + demand)
print('(작업자) W = ', [model.W[t]() for t in T])
print('(고용) H = ', [model.H[t]() for t in T])
print('(해고) L = ', [model.L[t]() for t in T])
print('(생산) P = ', [model.P[t]() for t in T])
print('(재고) I = ', [model.I[t]() for t in T])
print('(부재고) S = ', [model.S[t]() for t in T])
print('(외주) C = ', [model.C[t]() for t in T])
print('(잔업) O = ', [model.O[t]() for t in T])

Minimal Cost =  335760.0
(수요) D =  [0, 1600, 5000, 3200, 5800, 2200, 2200, 6500, 2300]
(작업자) W =  [80.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0]
(고용) H =  [None, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0]
(해고) L =  [None, 80.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
(생산) P =  [None, 600.0, 5000.0, 3200.0, 5800.0, 2200.0, 2200.0, 6500.0, 2800.0]
(재고) I =  [1000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 500.0]
(부재고) S =  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
(외주) C =  [None, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
(잔업) O =  [None, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 80.0]
